出于习惯，后续都用千问的模型来进行演示。
```bash
pip install -U langchain-qwq
```

In [66]:
from dotenv import load_dotenv
import os

load_dotenv()  # 自动查找当前目录的 .env 文件

dashscope_api_key = os.getenv("DASHSCOPE_API_KEY")
dashscope_base_url = os.getenv("DASHSCOPE_BASE_URL")


In [67]:
from langchain_qwq import ChatQwen

model = ChatQwen(
    model="qwen-plus",
    api_key=dashscope_api_key,
    base_url=dashscope_base_url,
    temperature=0.7,
)

# invoke

调用模型最直接的方法是使用invoke，传入单条消息或消息列表，其参数如下：

In [ ]:
response = model.invoke("介绍一下你自己")
print(response)

## RunnableConfig对象

config是很重要的参数，它接收一个 RunnableConfig 对象，用于配置模型的运行参数，控制模型的行为，例如设置温度、最大令牌数、停止序列等。

In [ ]:
from langchain_core.runnables import RunnableConfig

import inspect

# 查看定义的源码
print(inspect.getsource(RunnableConfig))

| 参数 | 类型 | 说明 |
|------|------|------|
| `tags` | `list[str]` | 标签，用于标记此调用及所有子调用（如 Chain 调用 LLM），以便后续在日志、监控面板（如 LangSmith）或分析工具中进行过滤和搜索|
| `metadata` | `dict[str, Any]` | 元数据，用于此调用及所有子调用，键应为字符串，值应为 JSON 可序列化 |
| `callbacks` | `Callbacks` | 回调函数，用于此调用及所有子调用，标签会传递给所有回调，元数据传递给 handle*Start 回调 |
| `run_name` | `str` | 追踪运行的名称，默认为类名 |
| `max_concurrency` | `int \| None` | 最大并行调用数，未提供则使用 `ThreadPoolExecutor` 的默认值 |
| `recursion_limit` | `int` | 最大递归调用次数，默认为 `25` |
| `configurable` | `dict[str, Any]` | 通过 `configurable_fields` 或 `configurable_alternatives` 预先设置为可配置属性的运行时值 |
| `run_id` | `uuid.UUID \| None` | 追踪运行的唯一标识符，未提供则自动生成新的 UUID |

随着课程的深入，以上参数的使用会逐步介绍，用得比较多的是configurable参数，用于配置模型的运行时参数，尤其是thread_id、checkpoint_id等参数，用于实现持久化存储、多轮对话等功能。

## 非init_chat_model添加configurable_fields

In [57]:
from langchain_core.runnables import ConfigurableField

config_model = model.configurable_fields(
    max_tokens=ConfigurableField(
        # id为键，与后续invoke时的config参数对应
        id="output_token_number",
        name="Max Tokens",
        description="最大令牌数"
    )
)

In [ ]:
response = config_model.invoke(
    "介绍一下你自己",
    config={
        "configurable": {
            "output_token_number": 32
            }
        }
    )
print(response)

## callbacks回调

指定一个或多个“回调处理器”给LangChain。当程序运行到特定阶段（如开始、结束、出错、打印日志）时，它会自动“回调”这些处理器，通知它们当前的状态：
1. 事件驱动：LangChain 定义了一系列事件（如 on_chain_start, on_chat_model_start, on_chat_model_new_token, on_tool_error 等）。
2. 注册：你在运行程序时，通过 config={"callbacks": [...]} 将处理器注册进去。
3. 触发：程序运行过程中，一旦触发了某个事件，LangChain 会自动遍历所有注册的处理器，并调用处理器中对应的方法。
4. 执行：处理器接收到事件通知后，执行你定义的逻辑（比如把日志存入数据库、发送告警、打印彩色日志等）。

### 自定义回调处理器

In [ ]:
from langchain_core.callbacks import BaseCallbackHandler
from datetime import datetime

# 1. 定义回调处理器，继承自BaseCallbackHandler
class MyCustomHandler(BaseCallbackHandler):
    def on_chat_model_start(self, serialized, prompts, **kwargs):
        # 当 LLM 开始运行时，这个方法会被自动调用
        print(f"[回调触发] LLM 启动于: {datetime.now()}")
        print(f"输入的提示词: {prompts}")

    def on_chat_model_end(self, response, **kwargs):
        # 当 LLM 结束运行时，这个方法会被自动调用
        print(f"[回调触发] LLM 结束于: {datetime.now()}")
        print(f"生成的文本长度: {len(response.generations[0][0].text)} 字符")


# 2. 运行时传入回调配置
result = model.invoke(
    "讲个关于猫的笑话",
    config={
        "callbacks": [MyCustomHandler()] # 注册你的处理器
    }
)

[回调触发] LLM 启动于: 2026-01-27 21:16:33.922031
输入的提示词: ['Human: 讲个关于猫的笑话']
[回调触发] LLM 结束于: 2026-01-27 21:16:39.007178
生成的文本长度: 249 字符


### 内置的回调处理器

利用回调callback的配置项来跨模型统计token

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.callbacks import get_usage_metadata_callback

model_1 = init_chat_model(model="gpt-4o-mini")
model_2 = init_chat_model(model="claude-haiku-4-5-20251001")

with get_usage_metadata_callback() as cb:
    # 使用 with 语句创建一个自动作用域，在这个作用域内，所有支持 usage metadata 的模型调用都会被自动监听
    # 本质上也是创建了一个 UsageMetadataCallbackHandler 实例，将其注册为全局回调上下文（通过 LangChain 的 callback_manager 机制），并返回一个handler在with块中使用，离开 with 块时自动清理上下文，避免污染后续调用。
    model_1.invoke("Hello")
    model_2.invoke("Hello")
    print(cb.usage_metadata)

# stream

大多数模型在生成输出内容的同时即可进行流式传输。通过逐步显示输出内容，流式传输能显著改善用户体验，尤其是对于较长的响应内容。
调用stream方法会返回一个迭代器，该迭代器会在输出块生成时立即产出，

invoke会在模型完成全部响应生成后才返回单个AIMessage，而stream方法会返回多个AIMessageChunk对象，每个对象都包含一部分输出文本，使用循环来实时处理每个数据块，最后通过加法运算合并成一个完整的消息。

In [ ]:
for chunk in model.stream("你好，介绍下你自己"):
    print(chunk.text, end="|", flush=True)

# batch

对模型批量处理一组相互独立的请求可以显著提升性能并降低成本，因为这些请求可以并行处理。

In [63]:
responses = model.batch([
    "为什么鹦鹉的羽毛是彩色的？",
    "飞机是如何飞行的？",
    "什么是量子计算？"
])
for response in responses:
    print(response)

content='鹦鹉羽毛呈现鲜艳多彩的颜色，是多种物理和生物机制共同作用的结果，主要包括以下两类原因：\n\n### 1. **结构色（Structural Color）——主要来源**\n这是鹦鹉色彩最独特、最重要的成因。  \n鹦鹉的羽毛中含有一种罕见的**类胡萝卜素结合蛋白**（如psittacofulvins），但更关键的是其羽毛微观结构能产生强烈的**结构色**：  \n- 羽毛羽枝（barbules）内部具有高度有序的纳米级**β-角蛋白球状结构**（类似微小的“光子晶体”）。  \n- 这些规则排列的纳米颗粒（直径约100–300 nm）能对特定波长的可见光产生**选择性散射、干涉和衍射**（类似肥皂泡或孔雀羽毛的原理）。  \n- 例如：蓝色并非来自蓝色色素（鹦鹉体内几乎不含蓝色色素），而是由短波长蓝光被纳米结构强烈散射所致；绿色则是**结构产生的蓝色 + 羽毛表层少量黄色类胡萝卜素色素叠加**而成（蓝+黄=绿）。\n\n✅ 这种结构色具有**虹彩效应**（随观察角度变化而变色）、**高饱和度**和**抗褪色性**（不因光照或时间轻易褪色），远超普通色素色。\n\n### 2. **色素色（Pigmentary Color）——辅助与调和**\n鹦鹉也使用天然色素，但种类有限且常与结构色协同：  \n- **类胡萝卜素（Carotenoids）**：从食物中摄取（如红辣椒、胡萝卜中的β-胡萝卜素、辣椒红素等），沉积在羽毛中产生**红色、橙色、黄色**。鹦鹉是少数能将黄色类胡萝卜素**代谢修饰为独特的红色酮基类胡萝卜素**（如astaxanthin衍生物）的鸟类，因此拥有鲜艳的红/橙色（如金刚鹦鹉的朱红胸羽）。  \n- **黑色素（Melanin）**：产生棕色、灰色、黑色及增强羽毛强度与耐久性（如飞羽的深色条纹）。  \n- ❌ *注意：鹦鹉体内基本不含* **蓝色或绿色色素**（如胆绿素在鸟类中极少用于羽毛着色），所以其蓝、绿、紫等色几乎全靠结构色实现。\n\n### ✅ 补充有趣的事实：\n- **演化意义**：鲜艳色彩有助于个体识别、配偶选择（性选择）、群体协调，某些拟态色（如部分亚马逊鹦鹉的绿色）也具隐蔽功能。  \n- **健康指示器**：羽毛色彩的鲜艳度可反映个体营养状况、寄生虫负荷和免疫状态（如类胡萝卜素需健康代谢才能有效沉积），是

默认情况下，batch() 仅返回整个批次的最终输出。如果你希望在每个输入完成生成时就立即收到其对应的输出，可以使用 batch_as_completed() 来流式获取结果。

<span style="color:red">使用batch_as_completed()时，结果可能会以乱序返回。每个结果都包含对应的输入索引，以便在需要时匹配并重新还原原始顺序。</span>

In [64]:
for response in model.batch_as_completed([
    "你好，介绍下你自己",
    "今天天气怎么样",
    "简单介绍一下transformer模型"
]):
    print(response)

(1, AIMessage(content='我无法实时获取当前天气信息，因为我的系统不连接网络，也无法访问地理位置或天气服务。\n\n不过你可以通过以下方式快速查看当地天气：\n\n✅ 手机：打开自带的「天气」App（如苹果天气、华为天气、小米天气等）  \n✅ 网页：访问中国气象局官网（www.cma.gov.cn）或常用平台（如墨迹天气、彩云天气、Weather.com）  \n✅ 语音助手：对 Siri / 小爱同学 / 小艺说：“今天天气怎么样？”  \n\n如果你告诉我你所在的城市（例如“北京”“广州”“成都”），我可以为你提供：\n🔹 当前季节的典型天气特征  \n🔹 近期穿衣/出行建议（如是否需带伞、防紫外线等）  \n🔹 有趣的天气小知识 🌧️☀️🌪️\n\n需要的话，随时告诉我你的城市吧 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 190, 'prompt_tokens': 11, 'total_tokens': 201, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'dashscope', 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-83adf689-1a3b-9bcc-9c24-292a4724a421', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bf54c-f5c1-70e0-8cfb-b3af6ae51e99-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 11, 'output_tokens': 190, 'total_tokens': 201, 'input_token_details': {'cache_read':

在使用batch或batch_as_completed处理大量输入时，可以通过在 RunnableConfig 字典中设置 max_concurrency 属性来控制最大并行调用数量。

以上方法均有异步版本，分别为ainvoke、astream、abatch、abatch_as_completed，随着课程的推进，再逐一详细介绍。